# define contextual files and DataFrames

In [6]:
from pandas import read_csv, DataFrame

ave_total_df = read_csv("metabolomics data/day_averaged_concentrations.csv")
ave_total_df.reset_index()
ave_total_df.index = [f'{row["diet"]}_{row["Rec_day_adj"]}' for index, row in ave_total_df.iterrows()]
ave_total_df.drop(["Rec_day_adj", "diet"], axis=1, inplace=True)
ave_total_df.drop([i for i in ave_total_df.index if "WD" in i or "PBS" in i], axis=0, inplace=True)

trials = list(ave_total_df.index)
print(trials)

unique_rec_day = list(map(float, {i.split("_")[1] for i in ave_total_df.index}))  # ['-3.0' '0.0' '11.0' '14.0' '28.0' '3.0' '5.0' '7.0']
print(unique_rec_day, type(unique_rec_day[0]))
def closest_number(number):
    closest = unique_rec_day[0]
    diff_val = abs(number-closest)
    for num in unique_rec_day:
        if abs(number-num) < diff_val:
            closest = num
            diff_val = abs(number-num)
        # elif abs(number-num) == diff_val:  print(closest, num)
    return closest

mega_metadata = read_csv("metabolomics data/a1_meta_c1-3.csv").set_index("seqID")
seqID_recDayAdj = {}
for index, row in mega_metadata.iterrows():
    if row["Rec_day_adj"] > 28 or float(row["Rec_day_adj"]) < -3:   continue
    seqID_recDayAdj[index] = f'{row["Treatment"]}_{closest_number(float(row["Rec_day_adj"]))}'
    # print(index, seqID_recDayAdj[index])
# print(mega_metadata["seqID"].to_list(), mega_metadata["Rec_day_adj"].to_list())
# seqID_recDayAdj = dict(zip([mega_metadata["seqID"].to_list(), mega_metadata["Rec_day_adj"].to_list()]))

print(mega_metadata["Rec_day_adj"].unique())
print({e.split("_")[1] for e in list(seqID_recDayAdj.values())})
display(mega_metadata)

['RC-ABX_-3.0', 'RC-ABX_0.0', 'RC-ABX_3.0', 'RC-ABX_5.0', 'RC-ABX_7.0', 'RC-ABX_11.0', 'RC-ABX_14.0', 'RC-ABX_28.0']
[5.0, 7.0, 14.0, 28.0, -3.0, 0.0, 3.0, 11.0] <class 'float'>
[-13.   -3.    0.    0.5   1.    1.5   4.    5.    6.    2.    3.    7.
   8.   11.   13.   14.   18.   22.   28.   36.   50.   64.  -31.   16.
  21.   23.   25. ]
{'5.0', '7.0', '14.0', '28.0', '-3.0', '0.0', '3.0', '11.0'}


,sampleID,Mouse,Treatment,Diet,Cage,Desc,Day,Rec_day,Rec_day_adj,Sac.group,Aim,Cohort,reads.in,reads.out,nseqs,seq_run,Abx,Post.diet,Post.micr
seqID,,,,,,,,,,,,,,,,,,,
MMF.16S.123_HF10,HF10,5264,WD-ABX,WD,4,baseline,0.0,-13.0,-13.0,0,A1,C1,226300,173866,157397,June_2021,ABX,NaN,NaN
MMF.16S.123_HF11,HF11,5265,WD-ABX,WD,4,baseline,0.0,-13.0,-13.0,0,A1,C1,130407,98088,81899,June_2021,ABX,NaN,NaN
MMF.16S.123_HF12,HF12,5732,WD-ABX,WD,4,baseline,0.0,-13.0,-13.0,0,A1,C1,152195,109574,93712,June_2021,ABX,NaN,NaN
MMF.16S.123_HF13,HF13,5735,RC-ABX,RC,1,post_diet,10.0,-3.0,-3.0,0,A1,C1,212831,155462,137121,June_2021,ABX,NaN,NaN
MMF.16S.123_HF14,HF14,5570,RC-ABX,RC,1,post_diet,10.0,-3.0,-3.0,0,A1,C1,37907,17565,15593,June_2021,ABX,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MMF.16S.202_HF1445,HF1445,1516,WD-ABX,WD,6072208,rec_wk4,77.0,25.0,25.0,4,A1,C3,85958,79831,61140,Feb_2022,ABX,NaN,NaN
MMF.16S.202_HF1446,HF1446,1507,RC-ABX,RC,6072209,rec_wk4,77.0,25.0,25.0,4,A1,C3,16807,15734,11936,Feb_2022,ABX,NaN,NaN
MMF.16S.202_HF1447,HF1447,1543,RC-ABX,RC,6072209,rec_wk4,77.0,25.0,25.0,4,A1,C3,54935,50100,40295,Feb_2022,ABX,NaN,NaN


# load and process the OTU file

In [10]:
from pandas import read_csv
# load the ASV aundances

# asv_abundances = asv_abundances if "asv_abundances" in locals() else read_csv("metabolomics data/merged_otu_table.txt", sep="\t").set_index("taxonID")
asv_abundances = read_csv("metabolomics data/merged_otu_table.txt", sep="\t").set_index("taxonID")
# display(asv_abundances.iloc[:20])
# asv_abundances.rename(columns=col_names, inplace=True)
# print(list(seqID_recDayAdj.keys()))
asv_abundances = asv_abundances.rename(columns=dict(seqID_recDayAdj)).drop([col for col in asv_abundances.columns if col not in list(seqID_recDayAdj.keys())], axis=1)
display(asv_abundances.iloc[:20])
# print([col for col in asv_abundances.columns if "MMF" in col])


# process the averaged 
averaged_asv_vals = asv_abundances.groupby(by=asv_abundances.columns, axis=1).mean().sort_index(axis=1)
averaged_asv_vals.to_csv("metabolomics data/averaged_asv_values.csv")
averaged_asv_vals.drop([col for col in averaged_asv_vals.columns if col not in ave_total_df.index], axis=1, inplace=True)
print(averaged_asv_vals.columns)
display(averaged_asv_vals.iloc[:20])
## merging identical ASVs
from json import load
with open("asvset_asvs.json", "r") as jsonIn:   asvSet = load(jsonIn)
for asv, SameASVs in asvSet.items():
    averaged_asv_vals.loc[asv] = sum([averaged_asv_vals.loc[identicalASV] for identicalASV in SameASVs])
    averaged_asv_vals.drop(SameASVs, axis=0, inplace=True)

transposed_averaged_asv_vals = averaged_asv_vals.T
trials = list(ave_total_df.index)
print(trials)

# developing the mapping dictionary of ASVs to their abundances
df_data= {}
for i, day_diet in enumerate(transposed_averaged_asv_vals.index):
    if day_diet not in trials or day_diet == 'RC-ABX_28.0': continue
    print(day_diet)
    trial_index = trials.index(day_diet)
    trial = day_diet.split("_")
    if trial[0] in trials[trial_index+1]:   future_trial = trials[trial_index+1]
    else:   continue

    trial_name = trial[0]+f"_{(float(trial[1])+float(future_trial.split('_')[1]))/2}"
    df_data[trial_name] = {column: (transposed_averaged_asv_vals.loc[future_trial].values[index]+value)/2
                           for index, (column, value) in enumerate(transposed_averaged_asv_vals.loc[day_diet].items())}

averaged_asv_vals_interday = DataFrame(df_data)
display(averaged_asv_vals_interday)

from json import dump
asv_dict = averaged_asv_vals_interday.to_dict(orient='index')
with open("metabolomics data/averaged_asv_abundances_interday_mapping.json", 'w') as jsonOut:
    dump(asv_dict, jsonOut, indent=3)

,RC-ABX_-3.0,RC-ABX_-3.0,RC-ABX_-3.0,RC-ABX_-3.0,RC-ABX_-3.0,RC-ABX_-3.0,WD-ABX_-3.0,WD-ABX_-3.0,WD-ABX_-3.0,WD-ABX_-3.0,WD-ABX_-3.0,WD-ABX_-3.0,RC-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,WD-ABX_0.0,WD-ABX_0.0,WD-ABX_0.0,WD-ABX_0.0,WD-ABX_0.0,WD-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,WD-ABX_0.0,WD-ABX_0.0,WD-ABX_0.0,WD-ABX_0.0,WD-ABX_0.0,WD-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,RC-ABX_0.0,...,WD-ABX_14.0,WD-ABX_14.0,WD-ABX_14.0,WD-ABX_14.0,RC-ABX_14.0,RC-ABX_14.0,RC-ABX_14.0,RC-ABX_14.0,RC-ABX_14.0,RC-ABX_14.0,WD-ABX_14.0,WD-ABX_14.0,WD-ABX_14.0,WD-ABX_14.0,WD-ABX_14.0,WD-ABX_14.0,RC-ABX_14.0,RC-ABX_14.0,RC-ABX_14.0,RC-ABX_14.0,WD-ABX_14.0,WD-ABX_14.0,WD-ABX_14.0,WD-ABX_14.0,RC-ABX_28.0,RC-ABX_28.0,RC-ABX_28.0,RC-ABX_28.0,WD-ABX_28.0,WD-ABX_28.0,WD-ABX_28.0,WD-ABX_28.0,RC-ABX_28.0,RC-ABX_28.0,RC-ABX_28.0,RC-ABX_28.0,WD-ABX_28.0,WD-ABX_28.0,WD-ABX_28.0,WD-ABX_28.0
taxonID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ASV00001;seqs=10697401;samples=1978,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ASV00001;seqs=15073372;samples=2336,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,63,1902,0,0,0,0,85,995,866,1043,0,0,0,0,297,162,4042,1856
ASV00002;seqs=7303020;samples=1403,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ASV00003;seqs=7068340;samples=1916,0,0,0,0,0,0,0,0,0,0,0,0,0,61,0,0,0,8,0,0,0,0,0,4,0,0,0,0,5,0,0,0,0,0,6,0,0,0,0,14,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ASV00003;seqs=8864566;samples=2185,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ASV00004;seqs=6473178;samples=1764,424,31,1232,302,297,4499,2051,7016,2526,1014,521,3895,28,913,0,0,0,18,20,24,0,50,7,16,13,0,0,0,14,19,37,0,0,54,194,6,0,0,0,0,...,4180,0,0,0,28,560,180,75,630,423,3489,2826,903,2874,304,53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ASV00004;seqs=7856150;samples=2507,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,254,2275,41,1167,0,0,0,0,26,321,126,109,0,0,0,0,0,59,164,130
ASV00005;seqs=5940024;samples=3018,0,0,0,0,0,0,0,0,0,0,0,0,15,89,0,0,16,54,2,0,0,2,0,9,7,3,0,0,0,0,0,0,5,11,16,0,0,4,0,21,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ASV00005;seqs=7246114;samples=3329,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Index(['RC-ABX_-3.0', 'RC-ABX_0.0', 'RC-ABX_11.0', 'RC-ABX_14.0',
       'RC-ABX_28.0', 'RC-ABX_3.0', 'RC-ABX_5.0', 'RC-ABX_7.0'],
      dtype='object')


,RC-ABX_-3.0,RC-ABX_0.0,RC-ABX_11.0,RC-ABX_14.0,RC-ABX_28.0,RC-ABX_3.0,RC-ABX_5.0,RC-ABX_7.0
taxonID,,,,,,,,
ASV00001;seqs=10697401;samples=1978,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,4.083333
ASV00001;seqs=15073372;samples=2336,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000
ASV00002;seqs=7303020;samples=1403,0.000000,1.250000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000
ASV00003;seqs=7068340;samples=1916,0.000000,17.500000,0.000000,0.0000,0.0,18.083333,0.000000,0.000000
ASV00003;seqs=8864566;samples=2185,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000
ASV00004;seqs=6473178;samples=1764,1130.833333,41.875000,624.000000,186.7500,0.0,13829.083333,18180.222222,487.000000
ASV00004;seqs=7856150;samples=2507,0.000000,0.000000,0.000000,0.1875,0.0,0.000000,0.000000,0.000000
ASV00005;seqs=5940024;samples=3018,0.000000,12.250000,0.000000,0.0000,0.0,8.083333,0.000000,0.000000
ASV00005;seqs=7246114;samples=3329,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000


['RC-ABX_-3.0', 'RC-ABX_0.0', 'RC-ABX_3.0', 'RC-ABX_5.0', 'RC-ABX_7.0', 'RC-ABX_11.0', 'RC-ABX_14.0', 'RC-ABX_28.0']
RC-ABX_-3.0
RC-ABX_0.0
